In [2]:
from __future__ import division, print_function

import numpy as np
import sympy as sym

from IPython.display import YouTubeVideo

In [3]:
YouTubeVideo('VqP2tREMvt0')

Continuing with vector spaces, especially the null and column spaces. Now we will learn how to compete these spaces and describe all of the vectors within these spaces. In other words, what's the algorithm for solving $\mathbf{A}x = 0$?

In [4]:
A = np.array([[1, 2, 2, 2],
              [2, 4, 6, 8],
              [3, 6, 8, 10]])

Problems we see immediately -- the second column is a multiple of the first column (it's not independent). Further, the third row is not independent, it's the sum of the first two rows. So now we need to use elimination, but need an algorith to do so even if there are zeros in the pivots.

Importantly -- while we are doing elimination to solve this system of equations (remember, solving for the *null space*), we're not changing that space. But we **will** be changing the column space.

We know the first pivot is A[0, 0] and we know the first step we need to take.

In [5]:
A_e = np.vstack([A[0], (A[1] - (2 * A[0])), (A[2] - 3 * A[0])])
A_e

array([[1, 2, 2, 2],
       [0, 0, 2, 4],
       [0, 0, 2, 4]])

In [6]:
A_e

array([[1, 2, 2, 2],
       [0, 0, 2, 4],
       [0, 0, 2, 4]])

But now there's a zero in the second pivot, and there's no non-zero below it for a row exchange. This tells us that that column *dependent* on the earlier columns! It's a combination of those columns. What about the third pivot?

In [7]:
U = np.vstack([A_e[0], A_e[1], (A_e[2] - A_e[1])])
U

array([[1, 2, 2, 2],
       [0, 0, 2, 4],
       [0, 0, 0, 0]])

We can call this **U** now, but it's not really upper triangular. It's in what's called *echelon form*. Strang says echelon form basically means "staircase" form. The non-zeros are not on the diagonal but form a kind of staircase.

So we only had two pivots to work with. More formally this means that the **rank** of the matrix is equal to 2. 

Now that we have $\mathbf{U}$, we can solve $\mathbf{U}x = 0$ using back substitution. What are the solutions?

We've separated out the *pivot columns* (variables), the first and the third columns. The other columns are called *free columns*. Why are they called free columns?

These columns correspond to $x_2$ and $x_4$ in our unknown coefficient matrix. We can assign anything we want to those two variables and solve the equations for $x_1$ and $x_3$. For instance, say we just assign them 1 and 0.

<center>
$x = 
\begin{bmatrix}
x_1 \\
1 \\
x_3 \\
0
\end{bmatrix}
$
</center>

In [8]:
np.linalg.matrix_rank(A)

2

Remember that our two equations, from the matrix above are:

<center>
$x_1 + 2x_2 + 2x_3 + 2x_4 = 0\\
2x_3 + 4x_4 = 0$
</center>

From this it's pretty clear that $x_3$ = 0, because we've set $x_4 = 0$.

If we've set $x_2$ to be 1, then we now know that $x_1 = -2$. So:

<center>
$x = 
\begin{bmatrix}
-2 \\
1 \\
0 \\
0
\end{bmatrix}
$
</center>

So, that's one vector in the null space. What are others? We could just take any multiple of $x$. 

<center>
$x = 
c \times
\begin{bmatrix}
-2 \\
1 \\
0 \\
0
\end{bmatrix}
$
</center>

What if we initialized $x_2$ and $x_4$ to be different values? Say 0 and 1 instead of 1 and 0, respectively.

<center>
$x = 
c \times
\begin{bmatrix}
2 \\
0 \\
-2 \\
1
\end{bmatrix}
$
</center>

We could take any combination of these two vectors as well. There's a solution for each free variable.

If the rank $r = 2$, then that's the number of pivot variables. Generalizing, for an $m \times n$ matrix with rank $r$, how many free variables are there? $n - r$ free variables.

This is a complete algorithm for finding all of the solutions to $\mathbf{A}x = 0$. Do elimination, ignoring columns that can't be operated on due to zeros, leaving $n - r$ free variables. 

One more step to think about. If **U** is in *upper echelon form*, we want to get it into *reduced row echelon form*, **R**.

In [9]:
U

array([[1, 2, 2, 2],
       [0, 0, 2, 4],
       [0, 0, 0, 0]])

What about the row of zeros? It's there because the third row was a combination of the first two, and this was revealed by elimination. Let's try doing elimination upwards and get zeros *above* **and** *below* the pivots.

In [10]:
U_r = np.vstack([(U[0] - U[1]), U[1], U[2]])
U_r

array([[ 1,  2,  0, -2],
       [ 0,  0,  2,  4],
       [ 0,  0,  0,  0]])

Now we want to make the pivots equal to 1 by dividing equation 2 by the pivot.

In [11]:
R = np.vstack([U_r[0], (U[1] / 2), U[2]])
R

array([[ 1.,  2.,  0., -2.],
       [ 0.,  0.,  1.,  2.],
       [ 0.,  0.,  0.,  0.]])

(You'll note here that Python converted these to floats because by default Python 3 no longer performs integer division.)

A matrix in reduced row echelon form contains a lot of information. We know the pivots are the first and third columns. It also contains the $2 \times 2$ identity matrix in the pivot rows and pivot columns. 

In [12]:
np.array_equal(np.eye(2), R[0:2, [0, 2]])

True

We can also find RREF directly as in Matlab using SymPy. The rref method can be called on a SymPy matrix and will return a tuple of $\mathbf{R}$, the matrix in RREF, and the pivot columns (zero-indexed).

In [14]:
sym.Matrix(A).rref()

(Matrix([
 [1.0, 2.0,   0, -2.0],
 [  0,   0, 1.0,  2.0],
 [  0,   0,   0,    0]]), [0, 2])

We can now also solve $\mathbf{R}x = 0$.

In fact, if we break **R** down into two components, the pivot components which form the identity matrix **I**, and the free components, which we'll call **F**, we actually see the two solutions we found for $x$ above with their signs switched as the columns!

$
\mathbf{I} = 
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$

$
\mathbf{F} = 
\begin{bmatrix}
2 & -2 \\
0 & 2
\end{bmatrix}
$

$
\mathbf{R} = 
\begin{bmatrix}
\mathbf{I} & \mathbf{F} \\
0 & 0
\end{bmatrix}
$

This is a matrix with $r$ pivot rows (remember, that's the *rank*), $r$ pivot columns, and $n - r$ free columns. 

If we want to solve $\mathbf{R}x = 0$, what are all the solutions at once? We need to create a *nullspace matrix*. The columns are "special solutions" from above. We want to find this nullspace matrix **N** such that:

$\mathbf{RN} = 0$.

It will work out such that:
$
\mathbf{N} = 
\begin{bmatrix}
-\mathbf{F} \\
\mathbf{I}
\end{bmatrix}
$

Note that while Matlab has convenient functions for finding the reduced row echelon form (rref) and nullspace matrix (null), NumPy does not. You can use `SymPy` to find it, or you can write your own function. For an interesting discussion about why this is, check [this out](http://mail.scipy.org/pipermail/numpy-discussion/2008-November/038705.html). As it turns out, you don't really have much cause to construct matrices in these way for most typical numerical computing.

In [40]:
A

array([[ 1,  2,  2,  2],
       [ 2,  4,  6,  8],
       [ 3,  6,  8, 10]])

Let's take a look at $\mathbf{A}'$.

In [16]:
A_t = A.T
A_t

array([[ 1,  2,  3],
       [ 2,  4,  6],
       [ 2,  6,  8],
       [ 2,  8, 10]])

How many pivot variables are there here? There are three columns, will there be three pivots? No. The third column is a combination of the first two. We'll have two pivots. We'll also find there are some dependent rows. What is the reduced row echelon form of $\mathbf{A}'$?

In [17]:
A_t_e = np.vstack([A_t[0], 
                  (A_t[1] - 2 * A_t[0]),
                  (A_t[2] - 2 * A_t[0]),
                  (A_t[3] - 2 * A_t[0])])
A_t_e

array([[1, 2, 3],
       [0, 0, 0],
       [0, 2, 2],
       [0, 4, 4]])

In [18]:
A_t_e_2 = np.vstack([A_t_e[0],
                     A_t_e[2],
                     A_t_e[1],
                     A_t_e[3]])

A_t_e_2

array([[1, 2, 3],
       [0, 2, 2],
       [0, 0, 0],
       [0, 4, 4]])

In [19]:
U = np.vstack([A_t_e_2[0],
               A_t_e_2[1],
               A_t_e_2[2],
               (A_t_e_2[3] - 2 * A_t_e_2[1])])

U

array([[1, 2, 3],
       [0, 2, 2],
       [0, 0, 0],
       [0, 0, 0]])

In [20]:
np.linalg.matrix_rank(U)

2

The rank is still 2 for the transpose of **A**. There is now *one* free column / variable. What's in the null space? What's $x$? Give the free variable a convenient variable (let's use 1). If we set it to 0, we'll only get zeros.

In [21]:
R = np.vstack([(U[0] - U[1]),
               (U[1] / 2),
               U[2],
               U[3]])

R

array([[ 1.,  0.,  1.],
       [ 0.,  1.,  1.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [22]:
sym.Matrix(A_t).rref() # in SymPy

(Matrix([
 [1.0,   0, 1.0],
 [  0, 1.0, 1.0],
 [  0,   0,   0],
 [  0,   0,   0]]), [0, 1])